## Step 1: Importing the necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.robust import mad
import math
import datetime
import json
import warnings
import re
warnings.filterwarnings('ignore')

## Step 2: Visits Data - Product (Target Variable) - US All Product (Web-App) Visit

### visits_product:

- Loads product visit data from multiple Excel files (spanning 2021 to 2024).
- Processes the data by converting date columns, filtering dates, and replacing certain values (like '-' with NaN).
- Combines data into a single DataFrame.

### visits_product_aggregate:

- Aggregates the product visit data by date, summing up visits and goals, and averaging the "% of Goal".
- Merges weather impact data back into the aggregated results and recalculates the "% of Goal".

### title_case:

- Converts a string to title case (capitalizing each word) and removes spaces.

### get_model_product_visits:

- Processes and aggregates product visit data by product and platform.
- Renames columns to indicate visits and goals per product.
- Merges product-level data with overall product visit aggregation.
- Ensures weather impact rating is properly formatted and merged with the result.


### Input Data:
- US All Product (Web-App) Visits 2021-10.17.23 - REVISED 1.4.24.xlsx - Contains Visits data from 2021-10-17 to 2023-10-01 (yyyy-mm-dd)
- US All Product (Web-App) Visits 10.1.23-12.31.23.xlsx - Contains Visits data from 2023-10-01 to 2023-12-31 (yyyy-mm-dd)
- US All Product (Web-App) Visits 1.1.24-5.31.24.xlsx - Contains Visits data from 2024-01-01 to 2024-05-31 (yyyy-mm-dd)
    

In [ ]:
def visits_product(file_path):
    """
    Loads and processes product visit data from multiple Excel files.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with processed product visit data
    """
    
    # Load data from Excel files
    df_visits_2021_to_2023 = pd.read_excel(file_path + "US All Product (Web-App) Visits 2021-10.17.23 - REVISED 1.4.24.xlsx")
    df_visits_2023_oct = pd.read_excel(file_path + "US All Product (Web-App) Visits 10.1.23-12.31.23.xlsx")
    df_visits_2024 = pd.read_excel(file_path + "US All Product (Web-App) Visits 1.1.24-5.31.24.xlsx")
    
    # Convert 'Date' column to datetime format
    df_visits_2021_to_2023['Date'] = pd.to_datetime(df_visits_2021_to_2023['Date'])
    df_visits_2023_oct['Date'] = pd.to_datetime(df_visits_2023_oct['Date'])
    df_visits_2024['Date'] = pd.to_datetime(df_visits_2024['Date'])
    
    # Filter data for df_v1 to include dates before 2023-10-01
    df_visits_2021_to_2023 = df_visits_2021_to_2023[df_visits_2021_to_2023['Date'] < '2023-10-01']

    # Concatenate the dataframes
    df_visits_product = pd.concat([df_visits_2021_to_2023, df_visits_2023_oct, df_visits_2024])
    
    # Replace '-' with NaN in relevant columns
    df_visits_product['% of Goal'] = df_visits_product['% of Goal'].replace('-', pd.NA)
    df_visits_product['US Weather Impact Rating'] = df_visits_product['US Weather Impact Rating'].replace('-', pd.NA)

    return df_visits_product

def visits_product_aggregate(file_path):
    """
    Aggregates product visit data by date.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with aggregated product visit data
    """
    
    df_visits_product = visits_product(file_path)
    
    # Group by 'Date' and aggregate values
    df_visits_grp = df_visits_product.groupby('Date').agg({
        'Value': 'sum',
        'Goal': 'sum',
        '% of Goal': 'mean'
    }).reset_index()

    # Merge the weather impact rating back into the grouped data
    df_visits_grp = pd.merge(
        df_visits_grp, 
        df_visits_product[['Date', 'US Weather Impact Rating']].drop_duplicates().sort_values('Date').reset_index(drop=True), 
        on='Date'
    )

    # Recalculate '% of Goal'
    df_visits_grp['% of Goal'] = df_visits_grp['Value'] / df_visits_grp['Goal']
    df_visits_grp['% of Goal'] = df_visits_grp['% of Goal'].replace([np.inf, -np.inf], 0)
    
    return df_visits_grp

def title_case(var):
    """
    Converts a string to title case while removing spaces.

    Parameters:
    - var: str, the string to be converted

    Returns:
    - str, the title-cased string with spaces removed
    """
    
    return ''.join(x for x in var.title() if not x.isspace())

def get_model_product_visits(file_path):
    """
    Processes and aggregates product visit data by product and overall.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with processed and aggregated product visit data
    """
    
    df_ = visits_product(file_path)
    df_['Date'] = pd.to_datetime(df_['Date'])
    
    df_visits = pd.DataFrame(columns=['Date'])

    # Process each unique product
    for ven in df_['Product'].unique():
        temp = df_[df_['Product'] == ven]
        var = title_case(temp['Platform'].values[0]) + "_" + ven
        
        # Group by 'Date', 'Platform', and 'Product' and sum the values
        df_visits_grp = temp.groupby(['Date', 'Platform', 'Product']).sum().reset_index()
        df_visits_grp = df_visits_grp.drop(['Platform', 'Product'], axis=1)
        
        # Rename columns
        df_visits_grp = df_visits_grp.rename({
            'Value': var + "_Visits", 
            'Goal': var + "_Goal"
        }, axis=1)
        
        # Merge with the result dataframe
        df_visits = pd.merge(df_visits, df_visits_grp, on='Date', how='outer')

    # Aggregate product visits and goals
    df_visits_grp = visits_product_aggregate(file_path)
    df_visits_grp = df_visits_grp.drop('% of Goal', axis=1)
    df_visits_grp = df_visits_grp.rename({
        'Value': "Overall_Product_Visits", 
        'Goal': "Overall_Product_Goal"
    }, axis=1)
    
    # Merge aggregated data into the result dataframe
    df_visits = pd.merge(df_visits, df_visits_grp, on='Date', how='outer')

    # Convert 'US Weather Impact Rating' to float
    df_visits['US Weather Impact Rating'] = df_visits['US Weather Impact Rating'].astype('float')
    
    return df_visits

## Step 3: Organic Search - Impressions/Clicks: Google Platform

### get_organic_search_google:

- Loads Google search impression data from three Excel files covering different date ranges.
- Combines the data from all three files into a single DataFrame.
- Converts the 'Date' column into a proper datetime format for consistency.

### Input Data:

- Search Impressions - Google Only - 6.18.22-10.17.23.xlsx - Contains Organic Search data from 2022-06-18 to 2023-10-17 (yyyy-mm-dd)
- Search Impressions - Google Only - 10.18.23-12.31.23 FIXED.xlsx - Contains Organic Search data from 2023-10-18 to 2023-12-31 (yyyy-mm-dd)
- Search Impressions - Google Only - 1.1.24-5.31.24.xlsx - Contains Organic Search data from 2024-01-01 to 2024-05-31 (yyyy-mm-dd)

In [ ]:

def get_organic_search_google(file_path):
    """
    Reads and concatenates Google search impression data from multiple Excel files.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with concatenated search impression data
    """
    
    # Read Excel files from different date ranges
    df_organic_2022_to_2023 = pd.read_excel(
        file_path + "Search Impressions - Google Only - 6.18.22-10.17.23.xlsx", 
        sheet_name='Dates'
    )
    df_organic_2023_oct = pd.read_excel(
        file_path + "Search Impressions - Google Only - 10.18.23-12.31.23 FIXED.xlsx", 
        sheet_name='Dates'
    )
    df_organic_2024 = pd.read_excel(
        file_path + "Search Impressions - Google Only - 1.1.24-5.31.24.xlsx", 
        sheet_name='Dates'
    )

    # Concatenate dataframes
    df_organic = pd.concat([df_organic_2022_to_2023, df_organic_2023_oct, df_organic_2024])

    # Convert 'Date' column to datetime format
    df_organic['Date'] = pd.to_datetime(df_organic['Date'])

    return df_organic

## Step 4: SEO Clicks - For Organic Search Clicks Imputation for missing data

### seo_pre_processing:

- Cleans column names and 'Segment' values: Strips spaces from column names and values.
- Merges the data: Combines SEO data from 2021, 2022, and 2023 on the 'Segment' column.
- Selects columns: Filters columns based on a date threshold ('2021-08-01').
- Transposes the data: Sets 'Segment' as the index, transposes the dataframe, and renames the 'index' column to 'Date'.

### seo_get_combined_column:

- Combines desktop and mobile SEO clicks: Adds the columns for "desktop" and "mobile web" SEO clicks into a single combined column.
- Selects relevant columns: Keeps only the 'Date' and the combined SEO clicks column.

### seo_clicks:

- Reads data from Excel files: Loads SEO click data for 2021, 2022, and 2023 from Excel files, skipping the first 6 rows.
- Processes the data: **Calls seo_pre_processing** to merge and clean the data, then **seo_get_combined_column** to aggregate the clicks.
- Returns the result: Outputs a DataFrame with combined desktop and mobile SEO clicks for the specified date range.

### Input Data
- US Web SEO Visits - 2021.xlsx - Contains SEO data of 2021 
- US Web SEO Visits - 2022.xlsx - Contains SEO data of 2022
- US Web SEO Visits - 2023 1.1-10.16.xlsx - Contains SEO data of 2023 

In [ ]:
def seo_pre_processing(df_seo_2021, df_seo_2022, df_seo_2023):
    """
    Pre-processes SEO data from multiple years by merging dataframes, 
    selecting columns based on a date filter, and transforming the data.

    Parameters:
    - df_seo_2021: DataFrame, SEO data for 2021
    - df_seo_2022: DataFrame, SEO data for 2022
    - df_seo_2023: DataFrame, SEO data for 2023

    Returns:
    - DataFrame with combined and processed SEO data
    """
    
    # Strip spaces from column names and 'Segment' values for all dataframes
    df_seo_2021.columns = df_seo_2021.columns.str.strip()
    df_seo_2021['Segment'] = df_seo_2021['Segment'].str.strip()
    
    df_seo_2022.columns = df_seo_2022.columns.str.strip()
    df_seo_2022['Segment'] = df_seo_2022['Segment'].str.strip()
    
    df_seo_2023.columns = df_seo_2023.columns.str.strip()
    df_seo_2023['Segment'] = df_seo_2023['Segment'].str.strip()
    
    # Merge the dataframes on 'Segment' column
    df_seo = pd.merge(df_seo_2021, df_seo_2022, on='Segment', how='outer')
    df_seo = pd.merge(df_seo, df_seo_2023, on='Segment', how='outer')

    # Select columns based on the date filter
    df_columns = df_seo.columns
    df_columns_to_include = df_columns[df_columns > '2021-08-01']
    df_seo = df_seo[df_columns_to_include]

    # Set 'Segment' as index and transpose the dataframe
    df_seo.set_index('Segment', inplace=True)
    df_seo = df_seo.T.reset_index()

    # Rename the 'index' column to 'Date'
    df_seo = df_seo.rename({'index': 'Date'}, axis=1)

    return df_seo

def seo_get_combined_column(df_seo):
    """
    Combines desktop and mobile SEO clicks into a single column.

    Parameters:
    - df_seo: DataFrame, SEO data with separate columns for desktop and mobile clicks

    Returns:
    - DataFrame with combined SEO clicks column
    """
    
    # Combine desktop and mobile SEO clicks into a single column
    df_seo['Desktop & Mobile SEO Clicks (Combined)'] = (
        df_seo['Organic Search; United States; desktop'] + 
        df_seo['Organic Search; United States; mobile web']
    )

    # Select only 'Date' and the combined SEO clicks column
    df_seo = df_seo[['Date', 'Desktop & Mobile SEO Clicks (Combined)']]

    return df_seo

def seo_clicks(file_path):
    """
    Reads and processes SEO click data from multiple Excel files.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with combined SEO clicks data
    """
    
    # Read data from Excel files, skipping the first 6 rows
    df_seo_2021 = pd.read_excel(file_path + "US Web SEO Visits - 2021.xlsx", skiprows=6)
    df_seo_2022 = pd.read_excel(file_path + "US Web SEO Visits - 2022.xlsx", skiprows=6)
    df_seo_2023 = pd.read_excel(file_path + "US Web SEO Visits - 2023 1.1-10.16.xlsx", skiprows=6)

    # Pre-process the data and get combined SEO clicks
    df_seo = seo_pre_processing(df_seo_2021, df_seo_2022, df_seo_2023)
    df_seo = seo_get_combined_column(df_seo)

    return df_seo

## Step 5: Pricing/App Installs (Android and iOS) - App Installs for Android and iOS

### pricing_android:

- Reads Android installation data from an Excel file.
- Processes the data by ensuring the 'Date' is in a consistent format and the 'Installs' column is of type float.
- Returns the processed DataFrame with Android installs.

### pricing_iOS:

- Reads iOS installation data from an Excel file.
- Similarly processes the 'Date' and 'Installs' columns to ensure consistency.
- Returns the processed DataFrame with iOS installs.

### pricing_aggregate:

- Processes and aggregates installation data for both Android and iOS by calling the previous two functions.
- Groups and sums installs by date for both platforms, then merges them into a single DataFrame.
- Adds a new column for total installs (Android + iOS) and returns the aggregated result.

### Input Data:
- Android Overview Dash Table Exported.xlsx - Contains Android Installs Data
- iOS Overview Dash Table Exported.xlsx - Contains iOS Installs Data

In [ ]:
def pricing_android(file_path):
    """
    Reads and processes Android installation data from an Excel file.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed Android install data
    """
    
    # Read the Android Overview data from the Excel file
    df_android_installs = pd.read_excel(file_path + "Android Overview Dash Table Exported.xlsx")
    
    # Convert the 'Date' column to string format (YYYY-MM-DD)
    df_android_installs['Date'] = pd.to_datetime(df_android_installs['Date']).dt.strftime('%Y-%m-%d')
    
    # Ensure the 'Installs' column is of type float
    df_android_installs['Installs'] = df_android_installs['Installs'].astype(float)
    
    # Convert the 'Date' column back to datetime format
    df_android_installs['Date'] = pd.to_datetime(df_android_installs['Date'])
    
    # Return the processed DataFrame for Android installs
    return df_android_installs

def pricing_iOS(file_path):
    """
    Reads and processes iOS installation data from an Excel file.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed iOS install data
    """
    
    # Read the iOS Overview data from the Excel file
    df_ios_installs = pd.read_excel(file_path + "iOS Overview Dash Table Exported.xlsx")
    
    # Convert the 'Date' column to string format (YYYY-MM-DD)
    df_ios_installs['Date'] = pd.to_datetime(df_ios_installs['Date']).dt.strftime('%Y-%m-%d')
    
    # Ensure the 'Installs' column is of type float
    df_ios_installs['Installs'] = df_ios_installs['Installs'].astype(float)
    
    # Convert the 'Date' column back to datetime format
    df_ios_installs['Date'] = pd.to_datetime(df_ios_installs['Date'])
    
    # Return the processed DataFrame for iOS installs
    return df_ios_installs

def pricing_aggregate(file_path):
    """
    Aggregates installation data for Android and iOS platforms.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with aggregated install data for both platforms
    """
    
    # Process Android and iOS install data
    df_android = pricing_android(file_path)
    df_iOS = pricing_iOS(file_path)

    # Group by 'Date' and sum the values for both platforms
    df_android = df_android.groupby('Date').sum()
    df_iOS = df_iOS.groupby('Date').sum()

    # Rename 'Installs' column to distinguish Android and iOS
    df_android = df_android.rename({'Installs': 'Android_Installs'}, axis=1)
    df_iOS = df_iOS.rename({'Installs': 'iOS_Installs'}, axis=1)

    # Merge the Android and iOS data on 'Date' and calculate total installs
    df_installs = pd.merge(df_android, df_iOS, on='Date', how='outer').reset_index()
    df_installs['Total_Installs'] = df_installs['Android_Installs'] + df_installs['iOS_Installs']

    return df_installs

## Step 6: User Acquisition - Media Spend, Impression and Clicks

### mkt_media_spend_pre_processing:
- This function preprocesses marketing media spend data according to the type of media sheet

- Converts Spend and Clicks columns from strings to floats, handling currency symbols and commas.
- Renames columns based on specific media sheets to ensure consistency.
- Standardizes date formats based on media sheet type, creating a new_date column with formatted dates.
- Renames and drops columns as needed, setting the appropriate date column name.


### mkt_media_spend_analyis:
- This function analyzes and aggregates marketing media spend data

- Loads data from various media sheets.
- Calls mkt_media_spend_pre_processing to clean and standardize data.
- Aggregates data by summing values for each date.
- Renames columns to reflect the source and sheet type for consistency.
- Combines aggregated data from all sheets into a single DataFrame.

### get_model_user_acquisition_mkt_media_spend:
- This function processes and aggregates user acquisition marketing media spend data from multiple sources:

- Specifies the list of media sources and their corresponding data source names.
- Calls mkt_media_spend_analyis to process and aggregate data for each media source.
- Returns the combined DataFrame containing aggregated marketing media spend data.

### Input Data:
- BL_TWC _ Marketing Spend Data.xlsx - Contains Paid Media Data for User Acquisition

In [ ]:
def mkt_media_spend_pre_processing(df_paid_media, paid_media_sheet):
    """
    Pre-processes marketing media spend data based on the media sheet type.

    Parameters:
    - df_paid_media: DataFrame, marketing media spend data
    - paid_media_sheet: str, type of media sheet to apply specific processing rules

    Returns:
    - df_paid_media: DataFrame, processed marketing media spend data
    - date_var: str, name of the date column in the processed DataFrame
    """
    
    # Convert 'Spend' column to float if it is in object format with currency symbols
    if df_paid_media['Spend'].dtype == 'object':
        df_paid_media['Spend'] = (df_paid_media['Spend']
                        .str.replace(',', '')
                        .str.replace('$', '')
                        .astype(float))
    
    # Rename 'clicks' to 'Clicks' if present
    if 'clicks' in df_paid_media.columns:
        df_paid_media = df_paid_media.rename({'clicks': 'Clicks'}, axis=1)
    
    # Clean and convert 'Clicks' column to float if in object format
    if 'Clicks' in df_paid_media.columns and df_paid_media['Clicks'].dtype == 'object':
        df_paid_media['Clicks'] = (df_paid_media['Clicks']
                         .replace({'\$': ''}, regex=True)
                         .replace({'\,': ''}, regex=True)
                         .astype(float))
    
    # Handle specific cases based on the media sheet
    if paid_media_sheet == 'IronSource Sonic':
        df_paid_media = df_paid_media.rename({'OS': 'event_date', 'Day': 'OS'}, axis=1)
    
    if paid_media_sheet == 'Persona.ly':
        df_paid_media = df_paid_media[~df_paid_media['Platform'].isna()]
    
    if 'event_date' in df_paid_media.columns:
        df_paid_media = df_paid_media.rename({'event_date': 'Day'}, axis=1)
    
    # Process dates for Persona.ly
    if paid_media_sheet == 'Persona.ly':
        df_paid_media['new_date'] = df_paid_media['Day']
        df_paid_media['format'] = 1
        
        # Identify and process date formats
        df_paid_media.loc[df_paid_media.Day.str.contains('/') == True, 'format'] = 2
        df_paid_media.loc[df_paid_media.format == 2, 'new_date'] = pd.to_datetime(
            df_paid_media.loc[df_paid_media.format == 2, 'Day'], format='%m/%d/%y').dt.strftime('%Y-%m-%d')
        df_paid_media.loc[df_paid_media.Date_Issue_flag == 1, 'new_date'] = pd.to_datetime(
            df_paid_media.loc[df_paid_media.Date_Issue_flag == 1, 'Day'], format='%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_paid_media.loc[df_paid_media.format == 1, 'new_date'] = pd.to_datetime(
            df_paid_media.loc[df_paid_media.format == 1, 'Day'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_paid_media.loc[df_paid_media.Date_Issue_flag == 1, 'new_date'] = pd.to_datetime(
            df_paid_media.loc[df_paid_media.Date_Issue_flag == 1, 'Day'], format='%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_paid_media = df_paid_media.drop('Date_Issue_flag', axis=1)
    
    # Process dates for other media sheets
    else:
        df_paid_media['format'] = 1
        
        # Identify and process date formats
        df_paid_media.loc[df_paid_media.Day.str.contains('/') == True, 'format'] = 2
        df_paid_media.loc[df_paid_media.format == 1, 'new_date'] = pd.to_datetime(
            df_paid_media.loc[df_paid_media.format == 1, 'Day'], format='%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        
        if paid_media_sheet == 'Bidease':
            pattern = r'\b\d{2}/\d{2}/\d{2}\b'
            df_paid_media.loc[((df_paid_media.format == 2) & (df_paid_media.Day.str.contains(pattern, regex=True))), 'format'] = 3
            df_paid_media.loc[df_paid_media.format == 2, 'new_date'] = pd.to_datetime(
                df_paid_media.loc[df_paid_media.format == 2, 'Day'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
            df_paid_media.loc[df_paid_media.format == 3, 'new_date'] = pd.to_datetime(
                df_paid_media.loc[df_paid_media.format == 3, 'Day'], format='%m/%d/%y').dt.strftime('%Y-%m-%d')
        else:
            df_paid_media.loc[df_paid_media.format == 2, 'new_date'] = pd.to_datetime(
                df_paid_media.loc[df_paid_media.format == 2, 'Day'], format='%m/%d/%y').dt.strftime('%Y-%m-%d')
    
    # Finalize date column renaming and drop unnecessary columns
    if 'event_date' in df_paid_media.columns:
        df_paid_media = df_paid_media.rename({'Day': 'event_date'}, axis=1)
        date_var = 'event_date'
    else:
        date_var = 'Day'
    
    df_paid_media[date_var] = df_paid_media['new_date']
    df_paid_media = df_paid_media.drop(['format', 'new_date'], axis=1)
    
    return df_paid_media, date_var

def mkt_media_spend_analyis(file_path, paid_media, data_source_dic):
    """
    Analyzes marketing media spend data by processing and aggregating data from different media sheets.

    Parameters:
    - file_path: str, path to the directory containing the Excel files
    - paid_media: list of str, names of the media sheets to process
    - data_source_dic: dict, mapping of media sheet names to data source names

    Returns:
    - DataFrame containing the aggregated marketing spend data
    """
    
    df_result = pd.DataFrame(columns=['Date'])
    
    for paid_media_sheet in paid_media:
        # Load the data from the appropriate sheet
        if paid_media_sheet == 'Persona.ly':
            df_paid_media = pd.read_excel(file_path + "BL_TWC _ Marketing Spend Data.xlsx", sheet_name='Persona.ly')
        else:
            df_paid_media = pd.read_excel(file_path + "BL_TWC _ Marketing Spend Data.xlsx", sheet_name=paid_media_sheet)
        
        # Pre-process the data
        df_paid_media, date_var = mkt_media_spend_pre_processing(df_paid_media, paid_media_sheet)
        
        # Rename columns for consistency
        df_paid_media = df_paid_media.rename({'Day': 'Date', 'event_date': 'Date'}, axis=1)
        
        # Existing commented code
        # platform = list(df_paid_media['Platform'].unique())
        # os_type = df_paid_media['OS'].unique()
        # temp_os_var = ''
        # if len(os_type) == 1:
        #     temp_os_var = '_Overall'
        # for os in os_type:
        #     df_grp = df_paid_media[df_['OS'] == os].reset_index(drop=True)
        #     df_grp['Date'] = pd.to_datetime(df_grp['Date'])
        #     df_grp = df_grp.groupby(['Date', 'Platform', 'OS']).sum().reset_index()
        #     df_grp = df_grp.drop(['Platform', 'OS'], axis=1)
        #     df_grp = df_grp.rename(columns={col: data_source_dic[paid_media_sheet] + "_" + paid_media_sheet + "_" + os + temp_os_var + "_" + col if col != 'Date' else col for col in df_grp.columns})
        #     df_grp.columns = df_grp.columns.str.replace(' ', '_')
        #     df_grp['Date'] = pd.to_datetime(df_grp['Date'])
        #     df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
        
        # if len(os_type) > 1:
        #     df_grp = df_paid_media.groupby('Date').sum().reset_index()
        #     df_grp['Date'] = pd.to_datetime(df_grp['Date'])
        #     df_grp = df_grp.rename(columns={col: data_source_dic[paid_media_sheet] + "_" + paid_media_sheet + "_Overall" + "_" + col if col != 'Date' else col for col in df_grp.columns})
        #     df_grp.columns = df_grp.columns.str.replace(' ', '_')
        #     df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
        
        # Aggregate data and rename columns
        df_grp = df_paid_media.groupby('Date').sum().reset_index()
        df_grp['Date'] = pd.to_datetime(df_grp['Date'])
        df_grp = df_grp.rename(columns={col: data_source_dic[paid_media_sheet] + "_" + paid_media_sheet + "_" + col if col != 'Date' else col for col in df_grp.columns})
        df_grp.columns = df_grp.columns.str.replace(' ', '_')
        
        # Merge aggregated data with the result DataFrame
        df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    
    return df_result

def get_model_user_acquisition_mkt_media_spend(file_path):
    """
    Processes and aggregates user acquisition marketing media spend data from multiple sources.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame containing the aggregated marketing media spend data
    """
    
    # Define the list of paid media sources to process
    paid_media = [
        'Google', 'IronSource Aura', 'Bidease', 'Digital Turbine',
        'Tapjoy', 'Apple Search Ads', 'LiftOff', 'IronSource Sonic',
        'Twitter', 'TikTok', 'Vibe', 'Persona.ly'
    ]
    
    # Define the mapping of paid media sources to their respective data source names
    data_source_dic = {
        'Google': 'SEM',
        'IronSource Aura': 'Preload',
        'Bidease': 'Programmatic',
        'Digital Turbine': 'Preload',
        'Tapjoy': 'Programmatic',
        'Apple Search Ads': 'SEM',
        'LiftOff': 'Programmatic',
        'IronSource Sonic': 'Programmatic',
        'Twitter': 'PaidSocial',
        'TikTok': 'Programmatic',
        'Vibe': 'PaidSocial',
        'Persona.ly': 'Programmatic'
    }
    
    # Analyze and aggregate marketing media spend data
    df_paid_media = mkt_media_spend_analyis(file_path, paid_media, data_source_dic)
    
    return df_paid_media

## Step 7: Brand - Media Spend, Impression (US Brand Basis)

### US_brand_media_spend_impression:

- Reads brand media spend and impression data from three Excel files covering different time periods.
- Concatenates the data from these files into a single DataFrame.
- Converts the 'Date' column to a datetime format.
- Returns the concatenated and processed data.

### get_model_brand_media_spend_impression:

- Uses the data from **US_brand_media_spend_impression** and prepares it for further analysis.
- Aggregates the data by date, summing up the total impressions and media spend for each day.
- Returns a DataFrame with the aggregated brand media impressions and spend, grouped by date.

### Input Data:

- US Brand - Basis - 2021-2023.xlsx -  Contains Brand Spend from 2021 to 2023-10-15
- US Brand - Basis - 10.16.23-12.31.23.xlsx -  Contains Brand Spend from 2023-10-16 to 2023-12-31
- TWC_MMM_Brand_Data File_01.01.24-05.31.24.xlsx -  Contains Brand Spend from 2024-01-01 to 2024-05-31

In [ ]:
def US_brand_media_spend_impression(file_path):
    """
    Reads and concatenates brand media spend and impression data from multiple Excel files.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with concatenated data from all files
    """
    
    # Read data from multiple Excel files
    df_brand_2021_to_2023 = pd.read_excel(file_path + "US Brand - Basis - 2021-2023.xlsx")
    df_brand_2023_oct = pd.read_excel(file_path + "US Brand - Basis - 10.16.23-12.31.23.xlsx")
    df_brand_2024 = pd.read_excel(file_path + "TWC_MMM_Brand_Data File_01.01.24-05.31.24.xlsx")
    
    # Concatenate data from all files
    df_brand = pd.concat([df_brand_2021_to_2023, df_brand_2023_oct])
    df_brand = pd.concat([df_brand, df_brand_2024])
    
    # Convert 'Date' column to datetime format
    df_brand['Date'] = pd.to_datetime(df_brand['Date'])
    
    return df_brand


def get_model_brand_media_spend_impression(file_path):
    """
    Processes and aggregates brand media spend and impression data.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with aggregated brand media spend and impression data
    """
    
    # Get concatenated data from the US_brand_media_spend_impression function
    df_ = US_brand_media_spend_impression(file_path)
    
    # Initialize an empty DataFrame for the results
    df_brand = pd.DataFrame(columns=['Date'])
    
    # Uncommented code for future use
    # for ven in df_['Vendor'].unique():
    #     temp = df_[df_['Vendor'] == ven]
    #     var = temp['Advertising Type'].values[0] + "_" + ven
    #     df_brand_grp = temp.groupby(['Date', 'Vendor', 'Advertising Type']).sum().reset_index()
    #     df_brand_grp = df_brand_grp.drop(['Vendor', 'Advertising Type'], axis=1)
    #     df_brand_grp = df_brand_grp.rename({'Impressions': var + "_Impressions", 'Spend': var + "_Spend"}, axis=1)
    #     df_brand = pd.merge(df_brand, df_brand_grp, on='Date', how='outer')
    
    # for ven in df_['Advertising Type'].unique():
    #     temp = df_[df_['Advertising Type'] == ven]
    #     var = temp['Advertising Type'].values[0]
    #     df_brand_grp = temp.groupby(['Date', 'Advertising Type']).sum().reset_index()
    #     df_brand_grp = df_brand_grp.drop(['Advertising Type'], axis=1)
    #     df_brand_grp = df_brand_grp.rename({'Impressions': var + "_Impressions", 'Spend': var + "_Spend"}, axis=1)
    #     df_brand = pd.merge(df_brand, df_brand_grp, on='Date', how='outer')
    
    # Group by 'Date' and aggregate the sum of 'Impressions' and 'Spend'
    df_brand_grp = df_.groupby(['Date']).sum().reset_index()
    df_brand_grp = df_brand_grp.rename({'Impressions': "Brand_Impressions", 'Spend': "Brand_Spend"}, axis=1)
    
    # Merge aggregated data with the result DataFrame
    df_brand = pd.merge(df_brand, df_brand_grp, on='Date', how='outer')
    
    return df_brand


## Step 8: Brand Health Measures - TWC Brand performance metrics

### get_brand_health_measures:

- Reads brand health measures data from a CSV file.
- Sets the 'Measure' column as the index and transposes the DataFrame so the measures become columns.
- Resets the index and renames the index column to 'Date' for further processing.
- Returns the processed DataFrame.

### get_year_qtr:

- Extracts the 'Year' and 'Quarter' from a 'Date' column formatted as 'YYYY-QX'.
- Adds 'Year' and 'Quarter' columns to the DataFrame for easier analysis.
- Returns the updated DataFrame with these new columns.

### get_model_brand_health_measures:

- Uses the **get_brand_health_measures** function to load and prepare brand health data.
- Adds missing quarters ('2021-Q2', '2023-Q4', and '2024-Q2') to the dataset.
- Extracts 'Year' and 'Quarter', sorts the data, and converts them into a datetime format.
- Resamples the data to a daily frequency, forward-filling missing values.
- Returns the final DataFrame with daily frequency for brand health measures.

### Input Data:
- Brand health measures 2021 - 2024.csv - Contains Brand Health Measures data from 2021 to 2024

In [ ]:
def get_brand_health_measures(file_path):
    """
    Reads and processes brand health measures data from a CSV file.

    Parameters:
    - file_path: str, path to the directory containing the CSV file

    Returns:
    - DataFrame with the transposed and indexed brand health measures data
    """
    
    # Read the CSV file into a DataFrame
    df_brand_health = pd.read_csv(file_path + "Brand health measures 2021 - 2024.csv")
    
    # Set 'Measure' column as index and transpose the DataFrame
    df_brand_health.set_index('Measure', inplace=True)
    df_brand_health = df_brand_health.T
    
    # Reset index and rename 'index' column to 'Date'
    df_brand_health = df_brand_health.reset_index()
    df_brand_health = df_brand_health.rename({'index': 'Date'}, axis=1)
    
    return df_brand_health


def get_year_qtr(df):
    """
    Extracts 'Year' and 'Quarter' from the 'Date' column and adds them as new columns.

    Parameters:
    - df: DataFrame, containing a 'Date' column in 'YYYY-QX' format

    Returns:
    - DataFrame with 'Year' and 'Quarter' columns added
    """
    
    # Regular expression pattern for extracting year and quarter
    pattern = re.compile(r'(\d+)-Q(\d+)')
    
    # Apply regex pattern to extract year and quarter into new columns
    df[['Year', 'Quarter']] = df['Date'].apply(lambda x: pd.Series(pattern.match(x).groups()))
    df['Quarter'] = df['Quarter'].astype(int)
    df['Year'] = df['Year'].astype(int)
    
    return df


def get_model_brand_health_measures(file_path):
    """
    Processes brand health measures data, adds missing quarters, and performs date resampling.

    Parameters:
    - file_path: str, path to the directory containing the CSV file

    Returns:
    - DataFrame with daily frequency and forward-filled values
    """
    
    # Get the initial brand health measures data
    df_brand_qtr = get_brand_health_measures(file_path)
    
    # Append missing quarters to the DataFrame
    df_brand_qtr = df_brand_qtr.append({'Date': '2021-Q2'}, ignore_index=True)
    df_brand_qtr = df_brand_qtr.append({'Date': '2023-Q4'}, ignore_index=True)
    df_brand_qtr = df_brand_qtr.append({'Date': '2024-Q2'}, ignore_index=True)
    
    # Extract 'Year' and 'Quarter' from 'Date' column
    df_brand_qtr = get_year_qtr(df_brand_qtr)
    
    # Sort DataFrame by 'Year' and 'Quarter'
    df_brand_qtr = df_brand_qtr.sort_values(['Year', 'Quarter'])
    
    # Convert 'Year' and 'Quarter' to a datetime index
    df_brand_qtr['Date_New'] = pd.to_datetime(df_brand_qtr['Year'].astype(str) + 'Q' + df_brand_qtr['Quarter'].astype(str))
    df_brand_qtr.set_index('Date_New', inplace=True)
    
    # Resample DataFrame to daily frequency and forward-fill missing values
    df_brand_health_measures = df_brand_qtr.resample('D').ffill()
    
    # Reset index and rename 'Date_New' to 'Date'
    df_brand_health_measures.reset_index(inplace=True)
    df_brand_health_measures = df_brand_health_measures.drop('Date', axis=1)
    df_brand_health_measures = df_brand_health_measures.rename({'Date_New': 'Date'}, axis=1)
    df_brand_health_measures['Date'] = pd.to_datetime(df_brand_health_measures['Date'])
    
    return df_brand_health_measures


## Step 9: Social Engagement - Scoial Media User engagement data

### social_engagement_pre_processing_khoros:

- Loads and processes social engagement data from Khoros.
- Cleans unwanted rows and columns, normalizes platform names, converts engagement metrics to float, and formats dates.
- Returns the processed Khoros data.

### social_engagement_pre_processing_sprinklr:

- Loads and processes social engagement data from Sprinklr.
- Cleans and filters rows, normalizes platform names, converts metrics to float, and formats the date column.
- Returns the processed Sprinklr data.

### social_engagement_pre_processing:

- Processes social engagement data from a general source.
- Renames columns, normalizes platform names, converts engagement metrics to float, aggregates data by date and platform, and fills missing values.
- Returns the processed data.

### get_model_social_engagement:

- Combines social engagement data from Khoros, Sprinklr, and the general source.
- Filters data by date, concatenates the datasets, and processes data for each platform.
- Aggregates and normalizes data, returning a DataFrame with combined social engagement data.

### Input Data:

- Social Engagements-2020-2023_Revised.xlsx - Contains Social Engagements data from  Khoros for 2020 to 2023 
- spr_web_analyst_with_Impressions_available_after_January_2022.xlsx - Contains Social Engagements data from Sprinklr for 2022 to 2023 
- social_engg.xlsx - Contains Social Engagements data from 2024-01-01 to 2024-05-31

In [ ]:
def social_engagement_pre_processing_khoros(file_path):
    """
    Processes social engagement data from Khoros.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed social engagement data from Khoros
    """
    
    # Load the data from the specified Excel sheet
    df_soc_eng_khrs = pd.read_excel(file_path + "Social Engagements-2020-2023_Revised.xlsx",
                        sheet_name='2020 - 2023 Oct 25th')
    
    # Remove unwanted rows and columns
    df_soc_eng_khrs = df_soc_eng_khrs[df_soc_eng_khrs['Outbound Post'] != 'Outbound Post']
    df_soc_eng_khrs = df_soc_eng_khrs[df_soc_eng_khrs['Impressions'] != 'Awareness']
    df_soc_eng_khrs = df_soc_eng_khrs[df_soc_eng_khrs['Total Engagements (SUM)'] != 'Consideration']
    df_soc_eng_khrs = df_soc_eng_khrs.drop(['Unnamed: 52'], axis=1)
    
    # Filter out rows with missing platform values
    df_soc_eng_khrs = df_soc_eng_khrs[~df_soc_eng_khrs['Platform'].isna()]
    
    # Normalize platform names
    df_soc_eng_khrs['Platform'] = np.where(df_soc_eng_khrs['Platform'] == 'Tiktok', 'TikTok', df_soc_eng_khrs['Platform'])
    df_soc_eng_khrs['Platform'] = np.where(df_soc_eng_khrs['Platform'] == 'Linkedin', 'LinkedIn', df_soc_eng_khrs['Platform'])
    df_soc_eng_khrs['Platform'] = np.where(df_soc_eng_khrs['Platform'] == 'Youtube', 'YouTube', df_soc_eng_khrs['Platform'])
    
    # Convert engagement metrics to float
    df_soc_eng_khrs['Estimated Clicks (SUM)'] = df_soc_eng_khrs['Estimated Clicks (SUM)'].astype('float')
    df_soc_eng_khrs['Impressions'] = df_soc_eng_khrs['Impressions'].astype('float')
    df_soc_eng_khrs['Total Engagements (SUM)'] = df_soc_eng_khrs['Total Engagements (SUM)'].astype('float')
    
    # Convert date column to datetime format
    df_soc_eng_khrs['Date'] = pd.to_datetime(df_soc_eng_khrs['Date'])
    
    # Filter for Khoros data source
    df_soc_eng_khrs = df_soc_eng_khrs[df_soc_eng_khrs['Source'] == 'Khoros']
    
    # Split and reformat date and time columns
    df_soc_eng_khrs[['Date_Copy', 'Time_Copy']] = df_soc_eng_khrs[['Date', 'Time']]
    df_soc_eng_khrs[['Date', 'Time']] = df_soc_eng_khrs['Date'].astype(str).str.split(' ', expand=True)
    df_soc_eng_khrs['Date'] = pd.to_datetime(df_soc_eng_khrs['Date'])
    
    return df_soc_eng_khrs


def social_engagement_pre_processing_sprinklr(file_path):
    """
    Processes social engagement data from Sprinklr.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed social engagement data from Sprinklr
    """
    
    # Load the data from the specified Excel file
    df_soc_eng_spr = pd.read_excel(file_path + "spr_web_analyst_with_Impressions_available_after_January_2022.xlsx")
    
    # Remove unwanted rows and columns
    df_soc_eng_spr = df_soc_eng_spr[df_soc_eng_spr['Outbound Post'] != 'Outbound Post']
    df_soc_eng_spr = df_soc_eng_spr[df_soc_eng_spr['Impressions'] != 'Awareness']
    df_soc_eng_spr = df_soc_eng_spr[df_soc_eng_spr['Total Engagements (SUM)'] != 'Consideration']
    df_soc_eng_spr = df_soc_eng_spr[~df_soc_eng_spr['Platform'].isna()]
    
    # Normalize platform names
    df_soc_eng_spr['Platform'] = np.where(df_soc_eng_spr['Platform'] == 'Tiktok', 'TikTok', df_soc_eng_spr['Platform'])
    df_soc_eng_spr['Platform'] = np.where(df_soc_eng_spr['Platform'] == 'Linkedin', 'LinkedIn', df_soc_eng_spr['Platform'])
    df_soc_eng_spr['Platform'] = np.where(df_soc_eng_spr['Platform'] == 'Youtube', 'YouTube', df_soc_eng_spr['Platform'])
    
    # Convert engagement metrics to float
    df_soc_eng_spr['Estimated Clicks (SUM)'] = df_soc_eng_spr['Estimated Clicks (SUM)'].astype('float')
    df_soc_eng_spr['Impressions'] = df_soc_eng_spr['Impressions'].astype('float')
    df_soc_eng_spr['Total Engagements (SUM)'] = df_soc_eng_spr['Total Engagements (SUM)'].astype('float')
    
    # Convert date column to datetime format
    df_soc_eng_spr['Date'] = pd.to_datetime(df_soc_eng_spr['Date'])
    
    # Filter for specific platforms and set data source
    df_soc_eng_spr = df_soc_eng_spr[df_soc_eng_spr['Platform'].isin(['Facebook', 'LinkedIn', 'YouTube', 'Instagram', 'Twitter', 'TikTok'])]
    df_soc_eng_spr['Source'] = 'Sprinklr'
    
    return df_soc_eng_spr


def social_engagement_pre_processing(file_path):
    """
    Processes social engagement data from a general source.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed social engagement data from the general source
    """
    
    # Load the data from the specified Excel file
    df_soc_eng_2024 = pd.read_excel(file_path + "social_engg.xlsx")
    
    # Rename columns for consistency
    df_soc_eng_2024.rename(columns={'Post Reach (SUM)': 'Impressions', 'Channel': 'Platform'}, inplace=True)
    
    # Normalize platform names
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'TIKTOK_BUSINESS', 'TikTok', df_soc_eng_2024['Platform'])
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'LINKEDIN_COMPANY', 'LinkedIn', df_soc_eng_2024['Platform'])
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'YOUTUBE', 'YouTube', df_soc_eng_2024['Platform'])
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'INSTAGRAM', 'Instagram', df_soc_eng_2024['Platform'])
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'FBPAGE', 'Facebook', df_soc_eng_2024['Platform'])
    df_soc_eng_2024['Platform'] = np.where(df_soc_eng_2024['Platform'] == 'TWITTER', 'Twitter', df_soc_eng_2024['Platform'])
    
    # Convert engagement metrics to float
    df_soc_eng_2024['Estimated Clicks (SUM)'] = df_soc_eng_2024['Estimated Clicks (SUM)'].astype('float')
    df_soc_eng_2024['Impressions'] = df_soc_eng_2024['Impressions'].astype('float')
    df_soc_eng_2024['Total Engagements (SUM)'] = df_soc_eng_2024['Total Engagements (SUM)'].astype('float')
    
    # Convert 'PublishedTime' to date and fill missing values
    df_soc_eng_2024['Date'] = pd.to_datetime(df_soc_eng_2024['PublishedTime']).dt.date
    df_soc_eng_2024 = df_soc_eng_2024[df_soc_eng_2024['Platform'].isin(['Facebook', 'LinkedIn', 'YouTube', 'Instagram', 'Twitter', 'TikTok'])]
    df_soc_eng_2024 = df_soc_eng_2024.fillna(0)
    
    # Aggregate data by date and platform
    df_soc_eng_2024 = df_soc_eng_2024.groupby(['Date', 'Platform']).agg({
        'Total Engagements (SUM)': 'sum',
        'Estimated Clicks (SUM)': 'sum',
        'Impressions': 'sum'
    }).reset_index()
    
    return df_soc_eng_2024


def get_model_social_engagement(file_path):
    """
    Combines and processes social engagement data from Khoros, Sprinklr, and a general source.

    Parameters:
    - file_path: str, path to the directory containing the Excel files

    Returns:
    - DataFrame with combined and processed social engagement data
    """
    
    # Get processed data from different sources
    df_khrs = social_engagement_pre_processing_khoros(file_path)
    df_spr = social_engagement_pre_processing_sprinklr(file_path)
    date_cutoff = pd.to_datetime('2023-12-31', format='%Y/%m/%d')
    
    # Filter data by date
    df_khrs = df_khrs[df_khrs['Date'] <= date_cutoff]
    df_spr = df_spr[df_spr['Date'] <= date_cutoff]
    
    # Get and process additional social engagement data
    df_socengg_2024 = social_engagement_pre_processing(file_path)
    
    # Concatenate data from different sources
    df_socengg = pd.concat([df_spr, df_khrs], axis=0)
    df_socengg = pd.concat([df_socengg, df_socengg_2024], axis=0)
    
    # Initialize result DataFrame
    df_social_engagement = pd.DataFrame(columns=['Date'])
    
    # Normalize platform names
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'Tiktok', 'TikTok', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'Linkedin', 'LinkedIn', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'Youtube', 'YouTube', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'TIKTOK_BUSINESS', 'TikTok', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'LINKEDIN_COMPANY', 'LinkedIn', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'YOUTUBE', 'YouTube', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'INSTAGRAM', 'Instagram', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'FBPAGE', 'Facebook', df_socengg['Platform'])
    df_socengg['Platform'] = np.where(df_socengg['Platform'] == 'TWITTER', 'Twitter', df_socengg['Platform'])
    
    # Process data for each platform
    for plt in df_socengg['Platform'].unique():
        temp = df_socengg[df_socengg['Platform'] == plt]
        df_soc_eng_grp = temp.groupby(['Date', 'Platform'])[['Impressions',
                                                     'Total Engagements (SUM)',
                                                     'Estimated Clicks (SUM)']].sum().reset_index()
        
        # Drop the 'Platform' column and rename columns
        df_soc_eng_grp = df_soc_eng_grp.drop(['Platform'], axis=1)
        df_soc_eng_grp = df_soc_eng_grp.rename(columns={
            col: "SocialEng_" + plt + "_" + col if col != 'Date' else col
            for col in df_soc_eng_grp.columns
        })
        df_soc_eng_grp.columns = df_soc_eng_grp.columns.str.rstrip('(SUM)').str.strip()
        df_soc_eng_grp.columns = df_soc_eng_grp.columns.str.replace(' ', '_')
        
        # Merge with the result DataFrame
        df_social_engagement = pd.merge(df_social_engagement, df_soc_eng_grp, on='Date', how='outer')
    
    return df_social_engagement

## Step 10: Marketing Events - TWC Campaign related data

### get_model_mkt_events:

- Reads a CSV file containing campaign/vendor event data.
- Converts the 'Start_Date' and 'End_Date' columns to datetime format.
- For each event, it creates a date range between the start and end date and marks those dates with a Campaign_Flag of 1.
- Filters the data based on a user-specified date range (ads_date_range).
- Groups the data by date and concatenates campaign names (if there are multiple events on the same date).
- Ensures all dates within the specified range are present, even if no event occurred on that day.
- Fills missing values in 'Campaign_Flag' with 0 and in 'Campaign/Vendor(s)' with "NA."

### Input Data:

- CampaignVendor_Events.csv - Contains TWC campaign data 

In [ ]:
def get_model_mkt_events(file_path, ads_date_range):
    """
    Processes marketing events data from a CSV file and prepares it for analysis.

    Parameters:
    - file_path: str, path to the directory containing the CSV file
    - ads_date_range: list, [start_date, end_date] specifying the date range for filtering

    Returns:
    - DataFrame with processed marketing events data
    """
    
    # Load the data from the CSV file
    df_mkt_events = pd.read_csv(file_path + "CampaignVendor_Events.csv")
    
    # Convert 'Start_Date' and 'End_Date' columns to datetime format
    df_mkt_events['Start_Date'] = pd.to_datetime(df_mkt_events['Start_Date'], format="%d-%m-%Y")
    df_mkt_events['End_Date'] = pd.to_datetime(df_mkt_events['End_Date'], format="%d-%m-%Y")
    
    # Initialize an empty DataFrame for storing campaign information
    df_campaign = pd.DataFrame(columns=['Date', 'Campaign_Flag', 'Campaign/Vendor(s)'])
    
    # Iterate through each row to create a date range for each campaign
    for idx, row in df_mkt_events.iterrows():
        campaign_name = row['Campaign/Vendor(s)']
        start_date = row['Start_Date']
        end_date = row['End_Date']
        
        # Create a DataFrame with dates from start_date to end_date
        df_date_range = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D'), columns=['Date'])
        df_date_range['Campaign_Flag'] = 1
        df_date_range['Campaign/Vendor(s)'] = campaign_name
        
        # Append the date range DataFrame to the main DataFrame
        df_campaign = pd.concat([df_campaign, df_date_range], ignore_index=True)
    
    # Fill missing values with 0 and "NA"
    df_campaign = df_campaign.fillna(0)
    
    # Filter DataFrame based on the provided date range
    df_campaign = df_campaign[(df_campaign['Date'] >= ads_date_range[0]) & (df_campaign['Date'] <= ads_date_range[1])]
    
    # Group by 'Date' and aggregate the results
    df_campaign = df_campaign.groupby('Date').agg({
        'Campaign_Flag': 'max',
        'Campaign/Vendor(s)': lambda x: ' ; '.join(x)
    }).reset_index()
    
    # Create a DataFrame with all dates within the specified range
    all_dates = pd.DataFrame(pd.date_range(start=ads_date_range[0], end=ads_date_range[1], freq='D'), columns=['Date'])
    
    # Merge with the all_dates DataFrame to include missing dates
    df_campaign = pd.merge(all_dates, df_campaign, on='Date', how='left')
    
    # Fill missing values with 0 for 'Campaign_Flag' and "NA" for 'Campaign/Vendor(s)'
    df_campaign['Campaign_Flag'] = df_campaign['Campaign_Flag'].fillna(0).astype(int)
    df_campaign['Campaign/Vendor(s)'] = df_campaign['Campaign/Vendor(s)'].fillna(pd.NA)

    return df_campaign

## Step 11: Critical Events - USA Critical Events data

### get_model_critical_events:

- Reads a CSV file containing critical event information.
- Converts the 'Start_Date' and 'End_Date' columns to datetime format.
- For each critical event, it creates a date range between the start and end date, marking those dates with a Critical_Event_Flag of 1 and recording the event name.
- Filters the data based on a user-specified date range (ads_date_range).
- Groups the data by date, and concatenates event names (if multiple events occur on the same day).
- Ensures all dates within the specified range are present, even if no event occurred on that day.
- Fills missing values in 'Critical_Event_Flag' with 0 and in 'Event_Name' with "NA."

### Input Data:

- Critical_Events.csv - Contains USA Critical Events data 

In [ ]:
def get_model_critical_events(file_path, ads_date_range):
    """
    Processes critical events data from a CSV file and prepares it for analysis.

    Parameters:
    - file_path: str, path to the directory containing the CSV file
    - ads_date_range: list, [start_date, end_date] specifying the date range for filtering

    Returns:
    - DataFrame with processed critical events data
    """
    
    # Load the data from the CSV file
    df_crit_events = pd.read_csv(file_path + "Critical_Events.csv")
    
    # Convert 'Start_Date' and 'End_Date' columns to datetime format
    df_crit_events['Start_Date'] = pd.to_datetime(df_crit_events['Start_Date'], format="%d-%m-%Y")
    df_crit_events['End_Date'] = pd.to_datetime(df_crit_events['End_Date'], format="%d-%m-%Y")
    
    # Initialize an empty DataFrame for storing critical event information
    df_event = pd.DataFrame(columns=['Date', 'Critical_Event_Flag', 'Event_Name'])
    
    # Iterate through each row to create a date range for each critical event
    for idx, row in df_crit_events.iterrows():
        event_name = row['Critical_Event'].split(': ')[1]  # Extract event name after ": "
        start_date = row['Start_Date']
        end_date = row['End_Date']
        
        # Create a DataFrame with dates from start_date to end_date
        df_date_range = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D'), columns=['Date'])
        df_date_range['Critical_Event_Flag'] = 1
        df_date_range['Event_Name'] = event_name
        
        # Append the date range DataFrame to the main DataFrame
        df_event = pd.concat([df_event, df_date_range], ignore_index=True)
    
    # Fill missing values with 0 for 'Critical_Event_Flag' and "NA" for 'Event_Name'
    df_event = df_event.fillna(0)
    
    # Filter DataFrame based on the provided date range
    df_event = df_event[(df_event['Date'] >= ads_date_range[0]) & (df_event['Date'] <= ads_date_range[1])]
    
    # Group by 'Date' and aggregate the results
    df_event = df_event.groupby('Date').agg({
        'Critical_Event_Flag': 'max',
        'Event_Name': lambda x: ' ; '.join(x)
    }).reset_index()
    
    # Create a DataFrame with all dates within the specified range
    all_dates = pd.DataFrame(pd.date_range(start=ads_date_range[0], end=ads_date_range[1], freq='D'), columns=['Date'])
    
    # Merge with the all_dates DataFrame to include missing dates
    df_event = pd.merge(all_dates, df_event, on='Date', how='left')
    
    # Fill missing values with 0 for 'Critical_Event_Flag' and "NA" for 'Event_Name'
    df_event['Critical_Event_Flag'] = df_event['Critical_Event_Flag'].fillna(0).astype(int)
    df_event['Event_Name'] = df_event['Event_Name'].fillna(pd.NA)

    return df_event

## Step 12: Influencer Data - Influencer Social Media Data

### get_model_influencer_data:

- Reads an Excel file containing influencer data from a specified sheet.
- Extracts three columns: 'Date', 'Influencer_Spend', and 'Influencer_Daily_Impressions'.
- Converts the 'Date' column to datetime format for easier manipulation and analysis.

### Input Data:

- Influencer_Data.xlsx - Contains Influencer data 

In [ ]:
def get_model_influencer_data(file_path):
    """
    Loads and processes influencer data from an Excel file.

    Parameters:
    - file_path: str, path to the directory containing the Excel file

    Returns:
    - DataFrame with processed influencer data containing 'Date', 'Influencer_Spend', and 'Influencer_Daily_Impressions'
    """
    
    # Load the influencer data from the specified sheet of the Excel file
    df_influencer = pd.read_excel(file_path + "Influencer_Data.xlsx", sheet_name='Influencer_data')
    
    # Select relevant columns from the DataFrame
    df_influencer = df_influencer[['Date', 'Influencer_Spend', 'Influencer_Daily_Impressions']]
    
    # Convert 'Date' column to datetime format
    df_influencer['Date'] = pd.to_datetime(df_influencer['Date'])
    
    return df_influencer

## Step 13: Merging all the tables

### get_model_data:

**Load and Process Data:**

- Product Visits: Loads product visits data and processes date information.
- Organic Search: Loads Google organic search data and renames columns.
- SEO Clicks: Loads SEO clicks data and renames columns.
- Pricing and Installs: Loads and renames pricing and app install data.
- User Acquisition: Loads user acquisition marketing data.
- Brand Media Spend: Loads brand media spend and impression data.
- Brand Health Measures: Loads brand health measures data.
- Social Engagement: Loads social engagement data.
- Marketing Events: Loads marketing events data and renames columns.
- Critical Events: Loads critical events data.
- Influencer: Loads influencer data

**Merge Data:**

- Merges all the processed data into a single DataFrame on the 'Date' column.

**Filter and Finalize:**

- Filters the final DataFrame by a specified date range.

**Returns:**

- Returns the merged and processed DataFrame containing data from all sources.

In [ ]:
def get_model_data(file_path):
    """
    Retrieves and processes model data from various sources, merging them into a single DataFrame.
    
    Parameters:
    - file_path: str, path to the directory containing the data files
    
    Returns:
    - df_aligned_data: DataFrame, merged and processed data from all sources
    """
    
    # Visits Data - Product (Target Variable) - US All Product (Web-App) Visits
    df_model_product_visits = get_model_product_visits(file_path)
    df_model_product_visits['Date'] = pd.to_datetime(df_model_product_visits['Date'])
    print("Product Visits", 
          df_model_product_visits.shape, 
          df_model_product_visits['Date'].min().strftime("%d-%m-%Y"), 
          df_model_product_visits['Date'].max().strftime("%d-%m-%Y"), 
          df_model_product_visits['Date'].max() - df_model_product_visits['Date'].min())
    
   
    # Organic Search - Impressions/Clicks: Google Platform
    df_model_organic_search = get_organic_search_google(file_path)
    df_model_organic_search = df_model_organic_search.rename(
        {'Clicks': 'OrganicSearch_Google_Clicks',
         'Impressions': 'OrganicSearch_Google_Impressions',
         'CTR': 'OrganicSearch_Google_CTR',
         'Position': 'OrganicSearch_Google_Position'}, 
        axis=1
    )
    print("Organic Search", 
          df_model_organic_search.shape, 
          df_model_organic_search['Date'].min().strftime("%d-%m-%Y"), 
          df_model_organic_search['Date'].max().strftime("%d-%m-%Y"), 
          df_model_organic_search['Date'].max() - df_model_organic_search['Date'].min())
    
    df_aligned_data = pd.merge(df_model_product_visits,
                             df_model_organic_search,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # SEO Clicks - For Organic Search Clicks Imputation for missing data
    df_model_seo_clicks = seo_clicks(file_path)
    df_model_seo_clicks = df_model_seo_clicks.rename(
        {'Organic Search; United States; desktop': 'SEO_Clicks_OrganicSearch_Desktop',
         'Organic Search; United States; mobile web': 'SEO_Clicks_OrganicSearch_MobileWeb',
         'Desktop & Mobile SEO Clicks (Combined)': 'SEO_Clicks_OrganicSearch_Desktop_MobileWeb(Combined)'}, 
        axis=1
    )
    df_model_seo_clicks['Date'] = pd.to_datetime(df_model_seo_clicks['Date'])
    print("SEO Clicks (Organic)", 
          df_model_seo_clicks.shape, 
          df_model_seo_clicks['Date'].min().strftime("%d-%m-%Y"), 
          df_model_seo_clicks['Date'].max().strftime("%d-%m-%Y"), 
          df_model_seo_clicks['Date'].max() - df_model_seo_clicks['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_seo_clicks,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Pricing/App Installs (Android and iOS)
    df_model_pricing_installs = pricing_aggregate(file_path)
    df_model_pricing_installs = df_model_pricing_installs.rename(
        columns={col: 'Pricing_' + col if col != 'Date' else col for col in df_model_pricing_installs.columns}
    )
    print("Pricing", 
          df_model_pricing_installs.shape, 
          df_model_pricing_installs['Date'].min().strftime("%d-%m-%Y"), 
          df_model_pricing_installs['Date'].max().strftime("%d-%m-%Y"), 
          df_model_pricing_installs['Date'].max() - df_model_pricing_installs['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_pricing_installs,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # User Acquisition - Media Spend, Impression, and Clicks
    df_model_mkt_media_spend = get_model_user_acquisition_mkt_media_spend(file_path)
    df_model_mkt_media_spend['Date'] = pd.to_datetime(df_model_mkt_media_spend['Date'])
    print("Media Spend Impression", 
          df_model_mkt_media_spend.shape, 
          df_model_mkt_media_spend['Date'].min().strftime("%d-%m-%Y"), 
          df_model_mkt_media_spend['Date'].max().strftime("%d-%m-%Y"), 
          df_model_mkt_media_spend['Date'].max() - df_model_mkt_media_spend['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_mkt_media_spend,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Brand - Media Spend, Impression (US Brand Basis)
    df_model_brand_media_spend_impression = get_model_brand_media_spend_impression(file_path)
    print("Brand Media Spend Impression (US Brand Basis)", 
          df_model_brand_media_spend_impression.shape, 
          df_model_brand_media_spend_impression['Date'].min().strftime("%d-%m-%Y"), 
          df_model_brand_media_spend_impression['Date'].max().strftime("%d-%m-%Y"), 
          df_model_brand_media_spend_impression['Date'].max() - df_model_brand_media_spend_impression['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_brand_media_spend_impression,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Brand Health Measures
    df_model_brand_health_measures = get_model_brand_health_measures(file_path)
    df_model_brand_health_measures = df_model_brand_health_measures[df_model_brand_health_measures['Date'] <= df_aligned_data['Date'].max()]
    print("Brand Measure", 
          df_model_brand_health_measures.shape, 
          df_model_brand_health_measures['Date'].min().strftime("%d-%m-%Y"), 
          df_model_brand_health_measures['Date'].max().strftime("%d-%m-%Y"), 
          df_model_brand_health_measures['Date'].max() - df_model_brand_health_measures['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_brand_health_measures,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Social Engagement
    df_model_social_engagemnet = get_model_social_engagement(file_path)
    print("Social Engagement", 
          df_model_social_engagemnet.shape, 
          df_model_social_engagemnet['Date'].min().strftime("%d-%m-%Y"), 
          df_model_social_engagemnet['Date'].max().strftime("%d-%m-%Y"), 
          df_model_social_engagemnet['Date'].max() - df_model_social_engagemnet['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_social_engagemnet,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    

    
    # Marketing Events
    df_model_mkt_events = get_model_mkt_events(file_path, [df_aligned_data['Date'].min(), df_aligned_data['Date'].max()])
    df_model_mkt_events = df_model_mkt_events.rename(
        columns={col: 'EventsCamp/Vend_' + col if col != 'Date' else col for col in df_model_mkt_events.columns}
    )
    print("Marketing Events", 
          df_model_mkt_events.shape, 
          df_model_mkt_events['Date'].min().strftime("%d-%m-%Y"), 
          df_model_mkt_events['Date'].max().strftime("%d-%m-%Y"), 
          df_model_mkt_events['Date'].max() - df_model_mkt_events['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_mkt_events,
                             on='Date',
                             how='left')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Critical Events
    df_model_critical_events = get_model_critical_events(file_path, [df_aligned_data['Date'].min(), df_aligned_data['Date'].max()])
    print("Critical Events", 
          df_model_critical_events.shape, 
          df_model_critical_events['Date'].min().strftime("%d-%m-%Y"), 
          df_model_critical_events['Date'].max().strftime("%d-%m-%Y"), 
          df_model_critical_events['Date'].max() - df_model_critical_events['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_critical_events,
                             on='Date',
                             how='left')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    


    # Influencer Data
    df_model_influencer_data = get_model_influencer_data(file_path)
    print("Influencer Data", 
          df_model_influencer_data.shape, 
          df_model_influencer_data['Date'].min().strftime("%d-%m-%Y"), 
          df_model_influencer_data['Date'].max().strftime("%d-%m-%Y"), 
          df_model_influencer_data['Date'].max() - df_model_influencer_data['Date'].min())
    
    df_aligned_data = pd.merge(df_aligned_data,
                             df_model_influencer_data,
                             on='Date',
                             how='outer')
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())



    # Filter data by date range
    df_aligned_data = df_aligned_data[(df_aligned_data['Date'] >= '2021-09-01') & (df_aligned_data['Date'] <= '2023-12-31')]
    print("ADS", 
          df_aligned_data.shape, 
          df_aligned_data['Date'].min().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max().strftime("%d-%m-%Y"), 
          df_aligned_data['Date'].max() - df_aligned_data['Date'].min())
    
    
    return df_aligned_data
